<h1 align="center"><b>Comprehensive Academic Program Monitoring & Evaluation System</b></h1>
<p align="center">By Juan Carlos Dominguez</p>

### Program Goals and Purpose  
The primary goal of our project was to design and develop a comprehensive system to evaluate degree programs within a university department. This system aimed to facilitate the collection, storage, and analysis of critical data to support ongoing program reviews. The solution needed to handle relational data, provide user-friendly interfaces, and accommodate future extensions such as new degree levels and evaluation methods.

### Team Collaboration and Development Approach  
This project was a collaborative effort where our team worked together on both the backend and frontend components. By leveraging our collective expertise, we ensured a seamless integration between the two layers of the application. The backend was developed using Flask and Python, which allowed efficient communication with the frontend, implemented using HTML and JavaScript.  

### Database Design and Implementation  
We designed and implemented the database using SQL to ensure a robust and scalable structure for managing relational data. The database was engineered to store information on degrees, courses, instructors, sections, goals, and evaluations. Relationships between these entities were carefully constructed to reflect the program's requirements, such as associating courses with multiple degrees and tracking semester-specific data for course sections.

### Key Features and Functionality  
Our system successfully supported the following core features:  
1. **Data Entry:**  
   - Input of degrees, courses, instructors, sections, and goals.  
   - Association of courses with program goals.  
   - Entry of semester-specific course and section data.  

2. **Evaluation Management:**  
   - Instructors could input evaluation data for each goal associated with their courses.  
   - Support for multi-degree course evaluations by duplicating data entry for multiple programs.  
   - Optional inclusion of improvement suggestions for subsequent semesters.  

3. **Querying Capabilities:**  
   - Retrieval of degree-specific information, such as associated courses, sections, and goals.  
   - Chronological listing of sections within a given time range.  
   - Status tracking for evaluation completion and percentage-based performance analysis.

### Achievements and Impact  
Our team successfully developed a scalable and intuitive system that met all the project requirements outlined in the program description. The database and application design ensured extensibility, allowing the system to adapt to future needs. By integrating powerful querying capabilities, we empowered users to gain meaningful insights into degree evaluations and performance trends. This project demonstrated our ability to integrate backend, frontend, and database components into a cohesive solution, addressing the department's current and future program evaluation needs.


# The following is the `University Schema` to hold our data:

In [ ]:
# Dept Table -- Departments Table: Stores information about academic departments.
CREATE TABLE Departments (
	dept_code VARCHAR(2) PRIMARY KEY, -- Unique code for each department (e.g., 'CS', 'EE').
    dept_name VARCHAR(100) -- Full name of the department (e.g., 'Computer Science').
);


# Degrees Table -- Degrees Table: Stores details of academic degrees offered by departments.
CREATE TABLE Degrees (
    degree_id INT AUTO_INCREMENT PRIMARY KEY, -- Unique ID for each degree.
    level VARCHAR(50), -- Level of the degree (e.g., 'BS', 'MS').
    degree_name VARCHAR(100), -- Name of the degree (e.g., 'Computer Science').
    dept_code VARCHAR(2), -- Department offering the degree (foreign key).
    FOREIGN KEY (dept_code) REFERENCES Departments(dept_code),
    UNIQUE(degree_name, level) -- Ensures unique combinations of degree name and level.
);


# Courses Table -- Courses Table: Stores information about courses offered by departments.
CREATE TABLE Courses (
    course_id VARCHAR(8) PRIMARY KEY, -- Unique identifier for the course (e.g., 'CS1010'). 
    course_name VARCHAR(100) NOT NULL, -- Full name of the course.
    core_class BOOLEAN, -- Indicates if the course is a core class.
    CHECK (course_id REGEXP '^[A-Z]{2,4}[0-9]{4}$'), -- Ensures course ID follows this specific format.
    dept_code VARCHAR(2), -- Department offering the course (foreign key).
    FOREIGN KEY (dept_code) REFERENCES Departments(dept_code)
);

# Goals Table -- Goals Table: Stores information about learning or program goals.
CREATE TABLE Goals (
    goal_code VARCHAR(4) PRIMARY KEY, -- Unique code for each goal (e.g., 'G001').
    goal_desc VARCHAR(255) -- Description of the goal.
); 

#Goal_Courses -- Goal_Courses Table: Maps courses to specific goals they address.
CREATE TABLE Goal_Courses (
	goal_code VARCHAR(4), -- Goal being addressed (foreign key).
    course_id VARCHAR(8), -- Course addressing the goal (foreign key).
    FOREIGN KEY (course_id) REFERENCES Courses(course_id),
    FOREIGN KEY (goal_code) REFERENCES Goals(goal_code),
    PRIMARY KEY (goal_code, course_id) -- Ensures unique course-goal relationships.
);


# Degree_Goals Relationship -- Degree_Goals Table: Maps goals to degrees, representing program-level objectives.
CREATE TABLE Degree_Goals (
    degree_id INT, -- Degree associated with the goal (foreign key).
    goal_code VARCHAR(4), -- Goal being addressed (foreign key).
    FOREIGN KEY (degree_id) REFERENCES Degrees(degree_id),
    FOREIGN KEY (goal_code) REFERENCES Goals(goal_code),
    PRIMARY KEY (degree_id, goal_code), -- Ensures unique degree-goal relationships.
    CONSTRAINT unique_goal_per_degree UNIQUE (degree_id, goal_code)  -- Validates uniqueness.
);


# Instructor Table -- Instructors Table: Stores information about instructors.
CREATE TABLE Instructors (
	instructor_id CHAR(8) NOT NULL, -- Unique identifier for each instructor.
    instructors_name VARCHAR(100) NOT NULL, -- Full name of the instructor.
    PRIMARY KEY (instructor_id),
    dept_code VARCHAR(2), -- Department the instructor belongs to (foreign key).
    FOREIGN KEY (dept_code) REFERENCES Departments(dept_code)
);

# Sections Table -- Sections Table: Stores information about course sections offered in different semesters.
CREATE TABLE Sections (
    semester ENUM('Spring', 'Summer', 'Fall') NOT NULL, -- Semester of the section. 
    year YEAR NOT NULL, -- Year of the section.
    sec_id CHAR(3) NOT NULL, -- Section identifier.
    course_id VARCHAR(8) NOT NULL, -- Course being offered (foreign key).
    instructor_id CHAR(8) NOT NULL, -- Instructor teaching the section (foreign key)
    num_of_students INT CHECK (num_of_students >= 0), -- Number of students in the section.
    PRIMARY KEY (semester, year, sec_id, course_id),  -- added course_id as part of primary key due to validation issues
    FOREIGN KEY (course_id) REFERENCES Courses(course_id), -- Unique identifier for a section. 
    FOREIGN KEY (instructor_id) REFERENCES Instructors(instructor_id) 
);

# Evaluations Table -- Evaluations Table: Stores evaluation details for assignments mapped to goals.
CREATE TABLE Evaluations (
    assignment VARCHAR(100), -- Name of the assignment (e.g. 'Homework', 'Quiz', 'Final Exam', etc).   
    instructor_id CHAR(8), -- Instructor associated with the evaluation (foreign key).
    sec_id CHAR(3) NOT NULL, -- Section ID where the evaluation took place.
    semester ENUM('Spring', 'Summer', 'Fall') NOT NULL, -- Semester of the evaluation.
    year YEAR NOT NULL, -- Year of the evaluation.
    course_id VARCHAR(8) NOT NULL, -- Course being evaluated (foreign key).
    goal_code VARCHAR(4), -- Goal being evaluated (foreign key)       
    Eval_txt VARCHAR(255), -- Evaluation text/feedback.       
    grades_A INT DEFAULT 0, -- Count of A grades assigned.      
    grades_B INT DEFAULT 0, -- Count of B grades assigned.       
    grades_C INT DEFAULT 0, -- Count of C grades assigned.     
    grades_F INT DEFAULT 0, -- Count of F grades assigned.      
    PRIMARY KEY (assignment, sec_id, semester, year, course_id, goal_code), -- Unique evaluation entry.
    FOREIGN KEY (semester, year, sec_id, course_id) REFERENCES Sections(semester, year, sec_id, course_id),
    FOREIGN KEY (course_id) REFERENCES Courses(course_id),
    FOREIGN KEY (instructor_id) REFERENCES Instructors(instructor_id),
    FOREIGN KEY (goal_code) REFERENCES Goals(goal_code)  
);

# Database Schema Description

This schema was designed to manage and evaluate academic programs at a university, supporting robust storage and relational analysis of key data entities such as departments, degrees, courses, instructors, goals, sections, and evaluations. Each table serves a specific purpose and is interconnected to enable efficient querying and reporting.

---

## **1. Departments Table (`Departments`)**
- **Purpose:** Stores information about academic departments offering degrees and courses.
- **Key Columns:**
  - `dept_code`: A unique 2-character identifier for each department (e.g., "CS").
  - `dept_name`: The full name of the department (e.g., "Computer Science").

---

## **2. Degrees Table (`Degrees`)**
- **Purpose:** Contains details of academic degrees offered by departments.
- **Key Columns:**
  - `degree_id`: A unique auto-incrementing identifier for each degree.
  - `level`: Indicates the level of the degree (e.g., "BS", "MS").
  - `degree_name`: Full name of the degree (e.g., "Computer Science").
  - `dept_code`: References the department offering the degree.
- **Features:** 
  - Enforces unique combinations of `degree_name` and `level`.

---

## **3. Courses Table (`Courses`)**
- **Purpose:** Stores details about courses offered by departments.
- **Key Columns:**
  - `course_id`: A unique identifier following a specified format (e.g., "CS1010").
  - `course_name`: The full name of the course.
  - `core_class`: Boolean indicating if the course is part of the core curriculum.
  - `dept_code`: References the department offering the course.

---

## **4. Goals Table (`Goals`)**
- **Purpose:** Maintains a list of learning or program goals.
- **Key Columns:**
  - `goal_code`: A unique 4-character code for each goal.
  - `goal_desc`: A descriptive text explaining the goal.

---

## **5. Goal-Courses Table (`Goal_Courses`)**
- **Purpose:** Maps program goals to the courses addressing them.
- **Key Columns:**
  - `goal_code`: References a goal from the `Goals` table.
  - `course_id`: References a course from the `Courses` table.
- **Features:** 
  - Ensures unique course-goal relationships.

---

## **6. Degree-Goals Table (`Degree_Goals`)**
- **Purpose:** Links program goals to specific degrees.
- **Key Columns:**
  - `degree_id`: References a degree from the `Degrees` table.
  - `goal_code`: References a goal from the `Goals` table.
- **Features:** 
  - Ensures unique goal assignments for each degree.

---

## **7. Instructors Table (`Instructors`)**
- **Purpose:** Stores information about instructors in various departments.
- **Key Columns:**
  - `instructor_id`: A unique 8-character identifier for each instructor.
  - `instructors_name`: Full name of the instructor.
  - `dept_code`: References the department the instructor belongs to.

---

## **8. Sections Table (`Sections`)**
- **Purpose:** Contains information about course sections offered in different semesters and years.
- **Key Columns:**
  - `semester`: The semester when the section is offered (e.g., "Fall").
  - `year`: The year of the section.
  - `sec_id`: A unique identifier for the section.
  - `course_id`: References the course being offered.
  - `instructor_id`: References the instructor teaching the section.
  - `num_of_students`: The number of students enrolled in the section.
- **Features:** 
  - Enforces unique section identifiers across semesters, years, and courses.

---

## **9. Evaluations Table (`Evaluations`)**
- **Purpose:** Tracks evaluations of goals in specific course sections.
- **Key Columns:**
  - `assignment`: The name of the assignment (e.g., "Homework").
  - `instructor_id`: References the instructor conducting the evaluation.
  - `sec_id`, `semester`, `year`: Reference the specific section being evaluated.
  - `course_id`: References the course related to the evaluation.
  - `goal_code`: References the goal being evaluated.
  - `Eval_txt`: Textual feedback for the evaluation.
  - `grades_A`, `grades_B`, `grades_C`, `grades_F`: Counts of grades assigned in the evaluation.
- **Features:** 
  - Ensures unique evaluations for a specific assignment in a course-section-goal combination.

---

## **Summary**
This schema provides a comprehensive framework for managing academic program data, enabling detailed tracking of relationships between departments, degrees, courses, goals, instructors, and evaluations. It ensures data consistency, supports extensibility for future requirements, and facilitates efficient reporting and analysis.
